In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#export
from exp.nb_01 import *
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
import torch.optim

???


In [3]:
mpl.rcParams['image.cmap'] = 'gray'

In [4]:
x_train, y_train,x_valid, y_valid = get_data()

In [5]:
n,m = x_train.shape
c =y_train.max()+1
nh = 50

In [6]:
class Model(nn.Module):
    def __init__(self, n_in,nh,n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in, nh),nn.ReLU(),nn.Linear(nh, n_out)]
        
    def __call__(self,x):
        for l in self.layers :x = l(x)
        return x
    

In [7]:
model = Model(m,nh,10)

In [8]:
pred = model(x_train)

In [9]:
pred.shape

torch.Size([50000, 10])

In [10]:
def log_softmax(x):return (x.exp()/(x.exp().sum(-1,keepdim=True))).log() #dim=-1

In [11]:
x=tensor([[1.,2.],[1.,2.],[1.,2.]])

In [12]:
log_softmax(x),log_softmax(x).shape

(tensor([[-1.3133, -0.3133],
         [-1.3133, -0.3133],
         [-1.3133, -0.3133]]), torch.Size([3, 2]))

In [13]:
x.shape

torch.Size([3, 2])

In [14]:
pred[0].sum(-1)

tensor(0.0224, grad_fn=<SumBackward2>)

In [15]:
y_train[:3]

tensor([5, 0, 4])

In [16]:
sm_pred = log_softmax(pred)

In [17]:
sm_pred[[0,1,2],[5,0,4]]

tensor([-2.1505, -2.1627, -2.3484], grad_fn=<IndexBackward>)

In [18]:
sm_pred[0]

tensor([-2.1617, -2.4505, -2.3873, -2.3705, -2.4696, -2.1505, -2.2217, -2.2905,
        -2.2910, -2.2882], grad_fn=<SelectBackward>)

In [19]:
def nll(input, target): return -input[range(target.shape[0]),target].mean()#negative log-likelihood

In [20]:
loss = nll(sm_pred, y_train)#!!!

In [21]:
loss

tensor(2.3155, grad_fn=<NegBackward>)

In [22]:
def log_softmax(x):return x-x.exp().sum(-1, keepdim = True).log()

In [23]:
test_near(nll(log_softmax(pred),y_train),loss)

In [24]:
def logsumexp(x):#防止数过大产生误差 偏移max
    m = x.max(-1)[0]
    return m+(x-m[:,None]).exp().sum(-1).log()

$$log\sum e^{x} = log(e^a \sum e^{x-a}) = a+log(\sum e^{x-a})$$

In [25]:
test_near(logsumexp(pred), pred.logsumexp(1))

In [26]:
def log_softmax(x):return x-x.logsumexp(-1,keepdim = True)

In [27]:
test_near(nll(log_softmax(pred),y_train), loss)

In [28]:
test_near(F.nll_loss(F.log_softmax(pred,1),y_train), loss)
test_near(F.cross_entropy(pred, y_train),loss)

In [29]:
loss_func=F.cross_entropy

In [30]:
#export
def accuracy(out,yb):return(torch.argmax(out,dim=1)==yb).float().mean()

In [31]:
bs =64
xb = x_train[0:bs]
preds = model(xb)
preds[0],preds.shape

(tensor([ 0.1487, -0.1402, -0.0769, -0.0601, -0.1592,  0.1599,  0.0887,  0.0199,
          0.0194,  0.0222], grad_fn=<SelectBackward>), torch.Size([64, 10]))

In [32]:
yb = y_train[0:bs]
loss_func(preds,yb)

tensor(2.3271, grad_fn=<NllLossBackward>)

In [33]:
accuracy(preds, yb)

tensor(0.0469)

In [34]:
lr = 0.5
epochs =1

In [35]:
for epoch in range(epochs):
    for i in range((n-1)//bs+1):
        start_i = i*bs
        end_i = start_i+bs
        xb = x_train[start_i:end_i]
        yb = y_train[start_i:end_i]
        loss =loss_func(model(xb),yb)
        loss.backward()
        with torch.no_grad():
            for l in model.layers:
                if hasattr(l,'weight'):
                    l.weight -= l.weight.grad*lr
                    l.bias -= l.bias.grad*lr
                    l.weight.grad.zero_()
                    l.bias.grad.zero_()
        

In [36]:
loss_func(model(xb),yb),accuracy(model(xb), yb)

(tensor(0.0578, grad_fn=<NllLossBackward>), tensor(1.))

In [37]:
#nn.Module.__setattr__
class Model(nn.Module):
    def __init__(self, n_in,nh,n_out):
        super().__init__()
        self.l1 =nn.Linear(n_in,nh)
        self.l2 =nn.Linear(nh,n_out)
    
    def __call__(self, x): return self.l2(F.relu(self.l1(x)))
        

In [38]:
model = Model(m, nh, 10)


In [39]:
model.l1

Linear(in_features=784, out_features=50, bias=True)

In [40]:
def fit():
    for epoch in range(epochs):
        for i in range((n-1)//bs + 1):
            start_i = i*bs
            end_i = start_i+bs
            xb = x_train[start_i:end_i]
            yb = y_train[start_i:end_i]
            loss = loss_func(model(xb), yb)

            loss.backward()
            with torch.no_grad():
                for p in model.parameters(): p -= p.grad * lr
                model.zero_grad()

In [41]:
fit()
loss_func(model(xb), yb), accuracy(model(xb), yb)

(tensor(0.2032, grad_fn=<NllLossBackward>), tensor(0.9375))

In [42]:
class DummyModule():
    def __init__(self, n_in, nh, n_out):
        self._modules = {}
        self.l1 = nn.Linear(n_in,nh)
        self.l2 = nn.Linear(nh,n_out)
        
    def __setattr__(self,k,v):
        if not k.startswith("_"): self._modules[k] = v
        super().__setattr__(k,v)
        
    def __repr__(self): return f'{self._modules}'
    
    def parameters(self):
        for l in self._modules.values():
            for p in l.parameters(): yield p

In [43]:
mdl = DummyModule(m,nh,10)
mdl

{'l1': Linear(in_features=784, out_features=50, bias=True), 'l2': Linear(in_features=50, out_features=10, bias=True)}

In [44]:
[o.shape for o in mdl.parameters()]


[torch.Size([50, 784]),
 torch.Size([50]),
 torch.Size([10, 50]),
 torch.Size([10])]

```python
for i,l in enumerate(self, layers): self.add_module(f'layer'_{i},l)  #等价于
self.layers = nn.ModuleList(layers)
```

In [45]:
model = nn.Sequential(nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh,10))


In [46]:
fit()
loss_func(model(xb), yb), accuracy(model(xb), yb)

(tensor(0.1058, grad_fn=<NllLossBackward>), tensor(1.))

In [47]:
class Optimizer():
    def __init__(self, params, lr = 0.5):
        self.params, self.lr = list(params),lr
    def step(self):
        with torch.no_grad():
            for p in self.params: p-=p.grad*lr
    def zero_grad(self):
        for p in self.params:p.grad.data.zero_()

The wrapper "with torch.no_grad()" temporarily set all the requires_grad flag to false. 

In [48]:
model = nn.Sequential(nn.Linear(m,nh), nn.ReLU(),nn.Linear(nh,10))

In [49]:
opt = Optimizer(model.parameters())

In [50]:
def fit():
    for epoch in range(epochs):
        for i in range((n-1)//bs + 1):
            start_i = i*bs
            end_i = start_i+bs
            xb = x_train[start_i:end_i]
            yb = y_train[start_i:end_i]
            loss = loss_func(model(xb), yb)

            loss.backward()
            opt.step()
            opt.zero_grad()

In [51]:
fit()
loss, acc =loss_func(model(xb), yb), accuracy(model(xb), yb)

In [52]:
torch.optim.SGD.step??

In [53]:
assert acc>0.7

# DataLoader

In [54]:
#export
class Dataset():
    def __init__(self,x,y):self.x, self.y =x ,y
    def __len__(self):return len(self.x)
    def __getitem__(self, i):return self.x[i],self.y[i]

In [55]:
train_ds,valid_ds = Dataset(x_train,y_train),Dataset(x_valid,y_valid)
assert len(train_ds)==len(x_train)

In [56]:
def get_model():
    model = nn.Sequential(nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh,10))
    return model, torch.optim.SGD(model.parameters(), lr=lr)

In [57]:
model,opt = get_model()

In [58]:
for epoch in range(epochs):
    for i in range((n-1)//bs + 1):
        xb,yb = train_ds[i*bs : i*bs+bs]
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

In [59]:
loss, acc =loss_func(model(xb), yb), accuracy(model(xb), yb)
loss, acc

(tensor(0.1166, grad_fn=<NllLossBackward>), tensor(0.9375))

In [60]:
class DataLoader():
    def __init__(self, ds, bs): self.ds,self.bs = ds,bs
    def __iter__(self):
        for i in range(0, len(self.ds), self.bs): yield self.ds[i:i+self.bs]

In [61]:
train_dl = DataLoader(train_ds, bs)
valid_dl = DataLoader(valid_ds, bs)

In [62]:
xb,yb=next(iter(valid_dl))

In [63]:
xb.shape,yb.shape

(torch.Size([64, 784]), torch.Size([64]))

In [64]:
#plt.imshow(xb[0].view(28,28))

In [65]:
model, opt = get_model()

In [66]:
def fit():
    for epoch in range(epochs):
        for xb,yb in train_dl:
            pred = model(xb)
            loss = loss_func(pred, yb)
            loss.backward()
            opt.step()
            opt.zero_grad()

In [67]:
fit()

In [68]:
loss, acc =loss_func(model(xb), yb), accuracy(model(xb), yb)
loss, acc

(tensor(0.0882, grad_fn=<NllLossBackward>), tensor(0.9688))

# Radom sampling

In [69]:
class Sampler():
    def __init__(self, ds, bs, shuffle = False):
        self.n, self.bs, self.shuffle = len(ds), bs, shuffle
    
    def __iter__(self):
        self.idxs = torch.randperm(self.n) if self.shuffle else torch.arange(self.n)
        for i in range(0,self.n,self.bs): yield self.idxs[i:i+self.bs]

In [70]:
small_ds = Dataset(*train_ds[:10])

In [71]:
s=Sampler(small_ds, 3,False)
[o for o in s]

[tensor([0, 1, 2]), tensor([3, 4, 5]), tensor([6, 7, 8]), tensor([9])]

In [72]:
s=Sampler(small_ds, 3,True)
[o for o in s]

[tensor([5, 3, 2]), tensor([4, 7, 6]), tensor([1, 0, 9]), tensor([8])]

In [73]:
def collate(b):
    xs,ys = zip(*b)
    return torch.stack(xs),torch.stack(ys)

class DataLoader():
    def __init__(self, ds, sampler, collate_fn=collate):
        self.ds,self.sampler,self.collate_fn = ds,sampler,collate_fn
        
    def __iter__(self):
        for s in self.sampler: yield self.collate_fn([self.ds[i] for i in s])

In [74]:
a=[tensor([1, 2, 3]), tensor([4, 5, 6]), tensor([7, 8, 9])]

In [75]:
torch.stack(a,dim = 1),torch.stack(a,dim = 0)

(tensor([[1, 4, 7],
         [2, 5, 8],
         [3, 6, 9]]), tensor([[1, 2, 3],
         [4, 5, 6],
         [7, 8, 9]]))

In [76]:
torch.stack(a,dim = 1).sum(dim=0)#0列1行

tensor([ 6, 15, 24])

In [77]:
#export
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler

In [78]:
train_dl = DataLoader(train_ds, bs, sampler=RandomSampler(train_ds), collate_fn=collate)
valid_dl = DataLoader(valid_ds, bs, sampler=SequentialSampler(valid_ds), collate_fn=collate)

In [79]:
model,opt = get_model()
fit()
loss_func(model(xb), yb), accuracy(model(xb), yb)

(tensor(0.2791, grad_fn=<NllLossBackward>), tensor(0.8906))

In [80]:
train_dl = DataLoader(train_ds, bs, shuffle=True, drop_last=True)
valid_dl = DataLoader(valid_ds, bs, shuffle=False)

In [81]:
model,opt = get_model()
fit()

loss,acc = loss_func(model(xb), yb), accuracy(model(xb), yb)
assert acc>0.7
loss,acc

(tensor(0.1144, grad_fn=<NllLossBackward>), tensor(0.9688))

In [82]:
def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    for epoch in range(epochs):
        # Handle batchnorm / dropout
        model.train()
#         print(model.training)
        for xb,yb in train_dl:
            loss = loss_func(model(xb), yb)
            loss.backward()
            opt.step()
            opt.zero_grad()

        model.eval()
#         print(model.training)
        with torch.no_grad():
            tot_loss,tot_acc = 0.,0.
            for xb,yb in valid_dl:
                pred = model(xb)
                tot_loss += loss_func(pred, yb)
                tot_acc  += accuracy (pred,yb)
        nv = len(valid_dl)
        print(epoch, tot_loss/nv, tot_acc/nv)
    return tot_loss/nv, tot_acc/nv

In [83]:
#export
def get_dls(train_ds, valid_ds, bs, **kwargs):
    return (DataLoader(train_ds, batch_size=bs, shuffle=True, **kwargs),
            DataLoader(valid_ds, batch_size=bs*2, **kwargs))

In [84]:
train_dl,valid_dl = get_dls(train_ds, valid_ds, bs)
model,opt = get_model()
loss,acc = fit(5, model, loss_func, opt, train_dl, valid_dl)

0 tensor(0.1890) tensor(0.9429)
1 tensor(0.1654) tensor(0.9489)
2 tensor(0.1406) tensor(0.9577)
3 tensor(0.0990) tensor(0.9694)
4 tensor(0.1028) tensor(0.9711)


In [85]:
assert acc>0.9


In [86]:
!python notebook2script.py 02_AI.ipynb


Converted 02_AI.ipynb to exp/nb_02.py
